In [11]:
# Установка необходимых библиотек (если еще не установлены)
!pip install lightgbm

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

# Загрузка данных
from google.colab import files
uploaded = files.upload()

# Предположим, что файлы названы train.csv и test.csv
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# Посмотрим на структуру
print(train.shape)
print(train.head())

# Целевая переменная
y = train['y']
X = train.drop(['y'], axis=1)

# Обнаружим категориальные признаки
categorical_features = X.select_dtypes(include=['object']).columns

# Инициализация и обучение OrdinalEncoder с обработкой новых категорий
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

# Обучение кодировщика на тренировочных данных
encoder.fit(X[categorical_features])

# Трансформация тренировочных данных
X[categorical_features] = encoder.transform(X[categorical_features])

# Разделение на тренировочную и валидационную выборки
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Обработка тестовых данных
test_ids = test['id']
X_test = test.drop(['id'], axis=1)

# Применение обученного кодировщика к тестовым данным
X_test[categorical_features] = encoder.transform(X_test[categorical_features])

# --- Добавленная часть --- #
# Обеспечим, что признаки в тестовых данных совпадают с признаками обучающей выборки
features_used = X.columns.tolist()

# Проверяем наличие признаков в X_test
missing_features = set(features_used) - set(X_test.columns)
if missing_features:
    print(f"В тестовых данных отсутствуют признаки: {missing_features}")
    for feature in missing_features:
        X_test[feature] = 0  # добавляем недостающие признаки с нулями

# Приводим тестовые данные к нужному порядку
X_test_aligned = X_test[features_used]
# --- Конец добавленной части -- #

# Обучение модели градиентного бустинга
model = lgb.LGBMClassifier(
    n_estimators=1000,
    learning_rate=0.05,
    class_weight='balanced',
    random_state=42
)

# Обучение с ранней остановкой через callback
model.fit(
    X_train,
    y_train,
    eval_set=[(X_valid, y_valid)],
    eval_metric='auc',
    callbacks=[lgb.early_stopping(stopping_rounds=50)]
)

# Оценка на валидационной выборке
valid_proba = model.predict_proba(X_valid)[:, 1]
print("ROC-AUC на валидации:", roc_auc_score(y_valid, valid_proba))

# Предсказание для тестовых данных
test_proba = model.predict_proba(X_test_aligned)[:, 1]

# Создание файла для подписки
submission = pd.DataFrame({
    'id': test_ids,
    'y': test_proba
})

# Сохранение файла
submission.to_csv('submission.csv', index=False)

print("Готово! Файл submission.csv создан.")

Saving test.csv to test (1).csv
Saving train.csv to train (1).csv
(28832, 21)
   id  age        job  marital    education  default housing loan    contact  \
0   0   56  housemaid  married     basic.4y       no      no   no  telephone   
1   1   57   services  married  high.school  unknown      no   no  telephone   
2   2   37   services  married  high.school       no     yes   no  telephone   
3   3   40     admin.  married     basic.6y       no      no   no  telephone   
4   4   56   services  married  high.school       no      no  yes  telephone   

  month  ... campaign  pdays  previous     poutcome emp.var.rate  \
0   may  ...        1    999         0  nonexistent          1.1   
1   may  ...        1    999         0  nonexistent          1.1   
2   may  ...        1    999         0  nonexistent          1.1   
3   may  ...        1    999         0  nonexistent          1.1   
4   may  ...        1    999         0  nonexistent          1.1   

   cons.price.idx  cons.conf.idx